In [2]:
def card_value(c):
    # Explicit mapping for every card rank
    mapping = {
        'A':14, 'K':13, 'Q':12, 'J':11, 'T':10,
        '9':9, '8':8, '7':7, '6':6, '5':5, '4':4, '3':3, '2':2
    }
    return mapping[c]

def hand_category_and_tiebreak(hand):
    # Determine the category and tie-break values for a given 5-card hand.
    from collections import Counter
    ranks = list(hand)
    counts = Counter(ranks)
    # Sort by frequency descending, then by card strength descending
    freq_items = sorted(counts.items(), key=lambda x: (x[1], card_value(x[0])), reverse=True)
    freq_values = [x[1] for x in freq_items]

    # Determine category rank
    # 5-of-a-kind: [5]
    # 4-of-a-kind: [4,1]
    # Full house: [3,2]
    # 3-of-a-kind: [3,1,1]
    # 2-pair: [2,2,1]
    # 1-pair: [2,1,1,1]
    # high card: [1,1,1,1,1]
    if freq_values == [5]:
        category = 7
    elif freq_values == [4, 1]:
        category = 6
    elif freq_values == [3, 2]:
        category = 5
    elif freq_values == [3, 1, 1]:
        category = 4
    elif freq_values == [2, 2, 1]:
        category = 3
    elif freq_values == [2, 1, 1, 1]:
        category = 2
    else:
        category = 1

    # Tie break:
    # Compare the cards as is from left to right by their strength.
    # We just convert the hand's characters directly.
    tie_values = tuple(card_value(c) for c in hand)

    return (category, tie_values)

if __name__ == "__main__":
    hands = []
    with open("input.txt", "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            hand_str = parts[0]
            bid = int(parts[1])
            cat, tvals = hand_category_and_tiebreak(hand_str)
            # We'll sort later based on category (descending) and tie values (descending)
            hands.append((hand_str, bid, cat, tvals))

    # Sort hands by category descending, then tie-values lexicographically descending
    # Strongest hand first
    hands.sort(key=lambda x: (x[2], x[3]), reverse=True)

    # Assign ranks
    # The weakest hand should get rank 1, so we reverse the logic:
    # strongest (index 0) should get rank = number_of_hands
    # weakest (last index) should get rank = 1
    n = len(hands)
    total = 0
    for i, (h, b, c, tv) in enumerate(hands):
        rank = n - i
        total += b * rank

    print(total)

250232501


In [3]:
from collections import Counter

def card_value(c):
    """ Map card rank to its value for comparison """
    mapping = {'A': 14, 'K': 13, 'Q': 12, 'J': 11, 'T': 10,
               '9': 9, '8': 8, '7': 7, '6': 6, '5': 5,
               '4': 4, '3': 3, '2': 2}
    return mapping[c]

def hand_category_and_tiebreak(hand):
    """ Determine hand category and tiebreak values """
    ranks = list(hand)
    counts = Counter(ranks)
    # Sort by frequency descending, then by card strength descending
    freq_items = sorted(counts.items(), key=lambda x: (x[1], card_value(x[0])), reverse=True)
    freq_values = [x[1] for x in freq_items]

    # Determine category rank
    if freq_values == [5]:
        category = 7  # Five of a Kind
    elif freq_values == [4, 1]:
        category = 6  # Four of a Kind
    elif freq_values == [3, 2]:
        category = 5  # Full House
    elif freq_values == [3, 1, 1]:
        category = 4  # Three of a Kind
    elif freq_values == [2, 2, 1]:
        category = 3  # Two Pair
    elif freq_values == [2, 1, 1, 1]:
        category = 2  # One Pair
    else:
        category = 1  # High Card

    tie_values = tuple(card_value(c) for c, _ in freq_items)
    return (category, tie_values)

def calculate_total_winnings(input_file):
    hands = []
    with open(input_file, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            hand_str = parts[0]
            bid = int(parts[1])
            cat, tvals = hand_category_and_tiebreak(hand_str)
            hands.append((hand_str, bid, cat, tvals))

    # Sort hands by category descending, then tie-values descending
    hands.sort(key=lambda x: (x[2], x[3]), reverse=True)

    # Calculate ranks and total winnings
    n = len(hands)
    total = 0
    for i, (h, b, c, tv) in enumerate(hands):
        rank = n - i
        total += b * rank

    return total

# Calculate winnings for the uploaded file
file_path = "input.txt"
total_winnings = calculate_total_winnings(file_path)
total_winnings

249661593

In [4]:
def card_value(c):
    # Explicit mapping for every card rank
    mapping = {
        'A': 14, 'K': 13, 'Q': 12, 'T': 10, '9': 9,
        '8': 8, '7': 7, '6': 6, '5': 5, '4': 4, '3': 3, '2': 2, 'J': 1  # Jokers (J) are the weakest
    }
    return mapping[c]

def hand_category_and_tiebreak(hand):
    # Determine the category and tie-break values for a given 5-card hand.
    from collections import Counter
    ranks = list(hand)
    joker_count = ranks.count('J')
    counts = Counter(r for r in ranks if r != 'J')
    # Sort by frequency descending, then by card strength descending
    freq_items = sorted(counts.items(), key=lambda x: (x[1], card_value(x[0])), reverse=True)
    freq_values = [x[1] for x in freq_items]

    # Add jokers to reach the strongest hand type possible
    if joker_count > 0:
        while joker_count > 0:
            if freq_values and freq_values[0] < 5:
                freq_values[0] += 1
            else:
                freq_values.append(1)
            freq_values.sort(reverse=True)
            joker_count -= 1

    # Determine category rank
    if freq_values == [5]:
        category = 7  # Five of a Kind
    elif freq_values == [4, 1]:
        category = 6  # Four of a Kind
    elif freq_values == [3, 2]:
        category = 5  # Full House
    elif freq_values == [3, 1, 1]:
        category = 4  # Three of a Kind
    elif freq_values == [2, 2, 1]:
        category = 3  # Two Pair
    elif freq_values == [2, 1, 1, 1]:
        category = 2  # One Pair
    else:
        category = 1  # High Card

    # Tie break:
    # For breaking ties, jokers are always treated as 'J'
    tie_values = tuple(card_value(c) for c in hand)
    return (category, tie_values)

if __name__ == "__main__":
    hands = []
    with open("sample-input.txt", "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            hand_str = parts[0]
            bid = int(parts[1])
            cat, tvals = hand_category_and_tiebreak(hand_str)
            # We'll sort later based on category (descending) and tie values (descending)
            hands.append((hand_str, bid, cat, tvals))

    # Sort hands by category descending, then tie-values lexicographically descending
    # Strongest hand first
    hands.sort(key=lambda x: (x[2], x[3]), reverse=True)

    # Assign ranks
    # The weakest hand should get rank 1, so we reverse the logic:
    # strongest (index 0) should get rank = number_of_hands
    # weakest (last index) should get rank = 1
    n = len(hands)
    total = 0
    for i, (h, b, c, tv) in enumerate(hands):
        rank = n - i
        total += b * rank

    print(total)

5905


In [5]:
def card_value(c):
    # Explicit mapping for every card rank
    mapping = {
        'A': 14, 'K': 13, 'Q': 12, 'T': 10, '9': 9,
        '8': 8, '7': 7, '6': 6, '5': 5, '4': 4, '3': 3, '2': 2, 'J': 1  # Jokers (J) are the weakest
    }
    return mapping[c]

def hand_category_and_tiebreak(hand):
    # Determine the category and tie-break values for a given 5-card hand.
    from collections import Counter
    ranks = list(hand)
    joker_count = ranks.count('J')
    counts = Counter(r for r in ranks if r != 'J')
    # Sort by frequency descending, then by card strength descending
    freq_items = sorted(counts.items(), key=lambda x: (x[1], card_value(x[0])), reverse=True)
    freq_values = [x[1] for x in freq_items]

    # Add jokers to reach the strongest hand type possible
    if joker_count > 0:
        while joker_count > 0:
            if freq_values and freq_values[0] < 5:
                freq_values[0] += 1
            else:
                freq_values.append(1)
            freq_values.sort(reverse=True)
            joker_count -= 1

    # Determine category rank
    if freq_values == [5]:
        category = 7  # Five of a Kind
    elif freq_values == [4, 1]:
        category = 6  # Four of a Kind
    elif freq_values == [3, 2]:
        category = 5  # Full House
    elif freq_values == [3, 1, 1]:
        category = 4  # Three of a Kind
    elif freq_values == [2, 2, 1]:
        category = 3  # Two Pair
    elif freq_values == [2, 1, 1, 1]:
        category = 2  # One Pair
    else:
        category = 1  # High Card

    # Tie break:
    # For breaking ties, jokers are always treated as 'J'
    tie_values = tuple(card_value(c) for c in hand)
    return (category, tie_values)

if __name__ == "__main__":
    hands = []
    with open("input.txt", "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            hand_str = parts[0]
            bid = int(parts[1])
            cat, tvals = hand_category_and_tiebreak(hand_str)
            # We'll sort later based on category (descending) and tie values (descending)
            hands.append((hand_str, bid, cat, tvals))

    # Sort hands by category descending, then tie-values lexicographically descending
    # Strongest hand first
    hands.sort(key=lambda x: (x[2], x[3]), reverse=True)

    # Assign ranks
    # The weakest hand should get rank 1, so we reverse the logic:
    # strongest (index 0) should get rank = number_of_hands
    # weakest (last index) should get rank = 1
    n = len(hands)
    total = 0
    for i, (h, b, c, tv) in enumerate(hands):
        rank = n - i
        total += b * rank

    print(total)

249138943
